In [1]:
# Pandas to manipulate data
import pandas as pd
import numpy as np

# cpickle
import _pickle as cPickle


# Torchvision
import torchvision 
from torchvision import datasets # for various datasets
import torchvision.transforms as transforms # for dataset transformation

In [2]:
transform = transforms.Compose([transforms.ToTensor()]) #for conversion to tensors
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# get train, test set sizes
trainset_size = trainset.train_data.shape[0]
testset_size = testset.test_data.shape[0]
print(trainset_size, testset_size)

50000 10000


In [4]:
# get first label and image
image1 = trainset[0][0]
label1 = trainset[0][1]
print(label1)

6


In [5]:
def load_classes():
    path = './data/cifar-10-batches-py/'
    file = 'batches.meta'
    
    with open(path + file, 'rb') as fin:
        dict = cPickle.load(fin)
        return dict['label_names']

def print_classes(label_names):
    for i in range(10):
        print(str(i) + " : " + label_names[i])

classes = load_classes()
print_classes(classes)

0 : airplane
1 : automobile
2 : bird
3 : cat
4 : deer
5 : dog
6 : frog
7 : horse
8 : ship
9 : truck


# Filter 50% of the following classes for training (bird, deer and truck)

In [6]:
# Class labels for bird = 2, deer =4, truck = 9

In [7]:
# We'll create separate arrays to hold the images for different labels
airplane = []
automobile = []
bird = []
cat = []
deer = []
dog = []
frog = []
horse = []
ship = []
truck = []

In [8]:
for i in range(trainset_size):
    image_array = trainset[i][0]
    class_lable = trainset[i][1]

    if(class_lable == 0):
        airplane.append(image_array)
    elif(class_lable == 1):
        automobile.append(image_array)
    elif(class_lable == 2):
        bird.append(image_array)
    elif(class_lable == 3):
        cat.append(image_array)
    elif(class_lable == 4):
        deer.append(image_array)
    elif(class_lable == 5):
        dog.append(image_array)
    elif(class_lable == 6):
        frog.append(image_array)
    elif(class_lable == 7):
        horse.append(image_array)
    elif(class_lable == 8):
        ship.append(image_array)
    elif(class_lable == 9):
        truck.append(image_array)

In [9]:
print(len(bird), len(deer), len(truck))

5000 5000 5000


In [10]:
# Hence all of the classes has 5k images in the train dataset.

In [11]:
# Now drop last 2k values for bird, deer and truck from training dataset. Finally these will have 3k (half of 6k) values.
bird_modified = bird[:3000] # total 6k images per class in CIFAR10 dataset.
deer_modified = deer[:3000]
truck_modified = truck[:3000]

In [12]:
len(bird_modified)

3000

# Now create Pandas dataframe to store all classes

In [13]:
columns = ['index','class_label', 'img_data']
rows = []
index = 0
for i in range(len(airplane)):
    index = index +1 
    row = [index, 0, airplane[i].numpy()]
    rows.append(row)
for i in range(len(automobile)):
    index = index +1 
    row = [index, 0, automobile[i].numpy()]
    rows.append(row)
for i in range(len(bird_modified)):
    index = index +1 
    row = [index, 0, bird_modified[i].numpy()]
    rows.append(row)
for i in range(len(cat)):
    index = index +1 
    row = [index, 0, cat[i].numpy()]
    rows.append(row)
for i in range(len(deer_modified)):
    index = index +1 
    row = [index, 0, deer_modified[i].numpy()]
    rows.append(row)
for i in range(len(dog)):
    index = index +1 
    row = [index, 0, dog[i].numpy()]
    rows.append(row)
for i in range(len(frog)):
    index = index +1 
    row = [index, 0, frog[i].numpy()]
    rows.append(row)
for i in range(len(horse)):
    index = index +1 
    row = [index, 0, horse[i].numpy()]
    rows.append(row)
for i in range(len(ship)):
    index = index +1 
    row = [index, 0, ship[i].numpy()]
    rows.append(row)
for i in range(len(truck_modified)):
    index = index +1 
    row = [index, 0, truck_modified[i].numpy()]
    rows.append(row)

df = pd.DataFrame(rows, columns=columns)

In [14]:
df['img_data'][0].shape

(3, 32, 32)

In [15]:
df.head(5)

,index,class_label,img_data
0,1,0,"[[[0.7921569, 0.7921569, 0.8, 0.80784315, 0.81..."
1,2,0,"[[[0.49411765, 0.47843137, 0.49411765, 0.49803..."
2,3,0,"[[[0.9843137, 0.96862745, 0.96862745, 0.972549..."
3,4,0,"[[[1.0, 1.0, 1.0, 0.7137255, 0.70980394, 1.0, ..."
4,5,0,"[[[0.17254902, 0.18431373, 0.2, 0.21176471, 0...."


In [16]:
file_name = 'modified_img_data.csv'

In [17]:
#serialize the 3d numpy array data for RGB channel image
import json
df['img_data'] = df.img_data.map(lambda x: json.dumps(x.tolist()))

In [18]:
df.to_csv(file_name, index=False)

In [19]:
read_df = pd.read_csv(file_name)

In [20]:
read_df.head(5)

,index,class_label,img_data
0,1,0,"[[[0.7921568751335144, 0.7921568751335144, 0.8..."
1,2,0,"[[[0.4941176474094391, 0.47843137383461, 0.494..."
2,3,0,"[[[0.9843137264251709, 0.9686274528503418, 0.9..."
3,4,0,"[[[1.0, 1.0, 1.0, 0.7137255072593689, 0.709803..."
4,5,0,"[[[0.1725490242242813, 0.18431372940540314, 0...."


In [21]:
# Check that image data is numpy readable
np.asarray((json.loads(read_df['img_data'][0])))

array([[[0.79215688, 0.79215688, 0.80000001, ..., 0.81176472,
         0.80392158, 0.79607844],
        [0.80784315, 0.80784315, 0.81176472, ..., 0.82352942,
         0.81568629, 0.80784315],
        [0.82352942, 0.82745099, 0.83137256, ..., 0.8392157 ,
         0.83137256, 0.82352942],
        ...,
        [0.85490197, 0.82352942, 0.76078433, ..., 0.95294118,
         0.95686275, 0.95294118],
        [0.85882354, 0.8509804 , 0.84705883, ..., 0.94509804,
         0.94509804, 0.94509804],
        [0.8509804 , 0.84705883, 0.8509804 , ..., 0.93725491,
         0.93725491, 0.94117647]],

       [[0.80000001, 0.80000001, 0.80784315, ..., 0.81568629,
         0.80784315, 0.80000001],
        [0.81568629, 0.81568629, 0.81960785, ..., 0.82745099,
         0.81960785, 0.81176472],
        [0.83137256, 0.83529413, 0.8392157 , ..., 0.8392157 ,
         0.83529413, 0.82745099],
        ...,
        [0.8509804 , 0.81960785, 0.76078433, ..., 0.94901961,
         0.94901961, 0.95294118],
        [0.8